In [219]:
import numpy as np

In [220]:
def fit(x_train,y_train):
    result = {}
    result["total_data"]=len(y_train)
    key_val = set(y_train)
    for key in key_val:
        result[key] = {}
        currentclassrow = (y_train==key)
        x_train_current = x_train[currentclassrow]
        y_train_current = y_train[currentclassrow]
        numberoffeatures = x_train.shape[1]
        for feature in range(1,1+numberoffeatures):
            result[key][feature]={}
            val = set(x_train[:,feature-1])
            for indval in val:
                result[key][feature][indval] = (x_train_current[:,feature-1]==indval).sum()
        result[key]["count"]=len(y_train_current)
    return result

In [221]:
def prob(dictionary,x,curr_class):
    output = np.log(1)
    #it has 2 components (prioir prob / class prob)
    classprob = np.log(dictionary[curr_class]["count"])-np.log(dictionary["total_data"])
    #now we do 2nd component
    numoffeatures = len(dictionary[curr_class].keys())-1
    for i in range(1,1+numoffeatures):
        xi = x[i-1]
        countwithxi = dictionary[curr_class][i][xi]+1
        prob = np.log(countwithxi)-np.log(dictionary[curr_class]["count"]+len(dictionary[curr_class][i].keys()))
        output +=  prob
    #print (output,classprob)
    return output+classprob

In [222]:
def predictsinglepoint(dictionary,x):
    classes = dictionary.keys()
    bestprob = -1000
    bestclass = -1
    for curr_class in classes:
        if(curr_class== "total_data"):
            continue
        p_curr_class = prob(dictionary, x, curr_class)
        if(p_curr_class>bestprob):
            bestprob = p_curr_class
            bestclass= curr_class
    return bestclass

In [223]:
def predict(dictionary , x_test):
    y_pred = []
    for x in x_test:
        x_class = predictsinglepoint(dictionary,x)
        y_pred.append(x_class)
        #print(x_class)
    return y_pred

In [224]:
def makelabel(column):
    seclim = column.mean()
    firlim = seclim / 2
    thirlim = firlim * 3 
    for i in range(len(column)):
        if(column[i]<firlim):
            column[i] = 0
        elif(column[i]>=firlim and column[i]<seclim):
            column[i]=1
        else:
            column[i]=2
    return column

In [225]:
from sklearn import datasets
db = datasets.load_iris()
x = db.data
y = db.target

In [226]:
for i in range(x.shape[-1]):
    x[:,i] = makelabel(x[:,i])

In [227]:
from sklearn import model_selection
x_train , x_test , y_train , y_test = model_selection.train_test_split(x,y)

In [228]:
print(x_train.shape)

(112, 4)


In [236]:
dictionary = fit(x_train,y_train)

In [237]:
print(dictionary[0])

{1: {1.0: 45, 2.0: 0}, 2: {1.0: 7, 2.0: 38}, 3: {0.0: 44, 1.0: 1, 2.0: 0}, 4: {0.0: 44, 1.0: 1, 2.0: 0}, 'count': 45}


In [238]:
y_pred = predict(dictionary,x_test)
print(y_pred)

[2, 0, 2, 1, 2, 0, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 0, 0, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 0, 2]


In [239]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       0.70      0.39      0.50        18
           2       0.52      0.80      0.63        15

   micro avg       0.63      0.63      0.63        38
   macro avg       0.74      0.73      0.71        38
weighted avg       0.67      0.63      0.62        38

[[ 5  0  0]
 [ 0  7 11]
 [ 0  3 12]]
